In [117]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from ML_utils import load_data,load_data_test_set, total_day_eve_night_grouping
df =pd.read_csv('train.csv')

### UNCOMMENT THIS PART TO GROUP FEATURES
feature_engineering = True
df=total_day_eve_night_grouping(df,grouping=feature_engineering)
exclude_list = [feature_name for feature_name in df.columns if feature_name not in['number_vmail_messages','number_customer_service_calls','churn']]
one_hot_on = True
normalize_on = True
oversample_on=True


###################
#### LOAD DATA ####

X_train,y_train,X_val,y_val,X_test,y_test,columns = load_data(df,exclude=exclude_list,one_hot=one_hot_on,normalize_=normalize_on,oversample=oversample_on)


### kernel not trick

# X_train=np.concatenate((X_train,X_train**3),axis=1)

# X_val=np.concatenate((X_val,X_val**3),axis=1)
# X_train.shape




(2975, 4)

In [118]:
from sklearn.svm import SVC
clf = SVC(kernel='linear')
clf.fit(X_train, y_train)
y_pred = clf.predict(X_val)

from sklearn.metrics import accuracy_score
accuracy_score(y_val, y_pred)

/home/2fps/miniconda3/envs/tf/lib/python3.9/site-packages/sklearn/utils/validation.py:1184: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


0.4662480376766091

In [119]:


def svm_function(x,y,reg=0.01):
    #initilizing weight
    w = np.zeros(x.shape[1])
    b = 0
    #initialize learning rate
    l_rate = 0.1
    #epoch
    epoch = 10000
    #output list

    #training svm
    for e in range(epoch):
        for i in range(x.shape[0]):
            
            val1 = np.dot(x[i], w) - b
            if (y[i]*val1 >= 1):
                dw=2*reg*w
                db=0
            else:
                dw=2*reg*w - x[i]*y[i]
                db=y[i]
                
            w = w - l_rate * dw
            b = b - l_rate * db
            
    
    
    return w,b



In [120]:
def predict (x,w,b):
    return np.sign(np.dot(x,w)+b)

y_pred=predict(X_val,w,b)

y_pred=np.sign(y_pred)


NameError: name 'w' is not defined

In [ ]:
y_pred.reshape(-1,1)

array([[-1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [-1.],
       [-1.],
       [ 1.],
       [-1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [-1.],
       [-1.],
       [ 1.],
       [-1.],
       [-1.],
       [-1.],
       [-1.],
       [-1.],
       [ 1.],
       [-1.],
       [ 1.],
       [-1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [-1.],
       [-1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [-1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [-1.],
       [-1.],
       [-1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [-1.],
       [-1.],
       [-1.],
       [ 1.],
       [-1.],
       [-1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [-1.],
       [ 1.],
       [ 1.],
       [ 1.],
      

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(y_val,y_pred)

0.10832025117739404